## Imports and Database Connections

In [1]:
from sqlalchemy import create_engine
from aws import utils
import pandas as pd

In [2]:
from jinja2 import Environment, FileSystemLoader
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template("base.html")

In [3]:
db_config = utils.get_secret()

engine_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user = db_config['username'],
    password = db_config['password'],
    host = db_config['host'],
    port = db_config['port'],
    database = db_config['dbname'],
)
engine = create_engine(engine_string)

## Field Map

In [ ]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.plotting import gmap

map_options = GMapOptions(lat=35.01562, lng=-85.3197, map_type="satellite", zoom=20, tilt=55)

# For GMaps to function, Google requires you obtain and enable an API key:
# https://developers.google.com/maps/documentation/javascript/get-api-key
# Replace the value below with your personal API key:
plot = gmap("<yourAPIkey>", map_options)

inspect_points = ColumnDataSource(data=dict(lat=[ 35.01562,  35.01555], lon=[-85.3197, -85.3185]))

plot.circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, source=inspect_points)

from bokeh.io import export_png
export_png(plot, filename="plot.png")

## Inspection Photos

In [5]:
file_id = 5
AWS_ACCESS_KEY_ID = 
AWS_SECRET_ACCESS_KEY = 
AWS_S3_RESULTS_BUCKET = 
photos = utils.read_det_file(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_S3_RESULTS_BUCKET, file_id, engine)
# construct photo tags
photos["FILE_LOC"]= "<img src='" + photos["FILE_LOC"] + "' width=88 height=66 />"

Successful S3 get_object response. Status - 200


## Header Section

In [6]:
sql_header=f"select video_detail.name as file_name, work_order, address, pipe_segment, operator, \
date as date_video_collected, description as model_type \
from video_detail \
left join video_project on video_project.file_id=video_detail.id \
left join video_modeltypes on video_project.model_type=video_modeltypes.code \
where file_id={file_id};"

conn = engine.connect()
header = pd.read_sql(sql_header, conn)
file_name = header["file_name"].values[0].replace('\n','')
work_order = header["work_order"].values[0].replace('\n','')
address = header["address"].values[0].replace('\n','')
pipe_segment = header["pipe_segment"].values[0].replace('\n','')
operator = header["operator"].values[0].replace('\n','')
date_video_collected = str(header["date_video_collected"].values[0])
model_type = header["model_type"].values[0].replace('\n','')

In [7]:
# map template vars for base.html
template_vars = {"title" : "Sewer Inspection Report", 
                 "insights": photos.to_html(index=False, escape=False),
                 "file_name": file_name,
                 "work_order": work_order,
                 "address": address,
                 "pipe_segment": pipe_segment,
                 "operator": operator,
                 "date_video_collected": date_video_collected,
                 "model_type": model_type             
                }

html_out = template.render(template_vars)

## Putting it All Together

In [8]:
from weasyprint import HTML
from weasyprint import default_url_fetcher
HTML(string=html_out, base_url="").write_pdf(target=f"{address}.pdf", stylesheets=["./bootstrap/css/styles.css", "./bootstrap/js/scripts.js"], presentational_hints=True, optimize_size=('fonts', 'images'))